In [1]:
import numpy as np
import pandas as pd
import dimarray as da
import shutil
path_greenland = '/home/beckmann/mnt/cluhome/exp_pism/my_first_runs/input_data/'
import os
sia_e =3.0
# sia_e =1.25

In [2]:
res = np.array([18000, 9000, 4500, 4500])
times = np.array([-125000, -25000, -5000, -1000])
test = times +1
y_end=times[1:]
y_end=np.append(y_end,0)
surface_sheme =['smb + ftt@margins', 'smb + ftt@margins', 'smb + ftt@margins','smb + ftt@margins' ]
ice_eq = ['SIA', 'SIA', 'SIA', 'SIA+SSA' ]
# cali = ['']*res.size
# cali[-3] = 'calibration'
d = {'year': times,'resolution': res,  'surface_sheme': surface_sheme, 'ice_equation': ice_eq}
df = pd.DataFrame(data =d)

In [3]:
df

,year,resolution,surface_sheme,ice_equation
0,-125000,18000,smb + ftt@margins,SIA
1,-25000,9000,smb + ftt@margins,SIA
2,-5000,4500,smb + ftt@margins,SIA
3,-1000,4500,smb + ftt@margins,SIA+SSA


In [4]:
def get_mx_my(d_inp,res):
    ''' input data, and wanted resolution for bootstrapping'''
    dx = d_inp.x[1]-d_inp.x[0]
    dy = d_inp.y[1]-d_inp.y[0]
    ly =( d_inp.y[-1]-d_inp.y[0]+dx)/2  #full grid length from corner to corner
    lx = (d_inp.x[-1]-d_inp.x[0]+dy)/2
    mx = (2*lx)/res
    my = (2*ly)/res
    
    return (mx,my)

d_inp = da.read_nc(path_greenland+'pism_Greenland_9000m_mcb_jpl_v_ctrl.nc')

skip = np.array([100,50,50,100])
Mz = ['-Mz 251 -Mbz 21'] *4
# Mz[-1] = '-Mz 501 -Mbz 41'
M_horizontal = []
time_command = []
time_command_test = []
#calutulate x and y length from resolution
for i,r in enumerate(res):
    mx,my = get_mx_my(d_inp,r)
    #print ('mx,', mx, 'my', my)
    M_horizontal.append('-Mx '+str(int(mx))+' -My '+str(int(my)))
    time_command.append('-ys '+str(int(times[i]))+ ' -ye '+str(int(y_end[i])))
    time_command_test.append('-ys '+str(int(times[i]))+ ' -ye '+str(int(test[i])))

d2 = {'year': times,'resolution': res,  '-Mx -My': M_horizontal, '-Mz': Mz, 'skip': skip, 'time command': time_command }
df2 = pd.DataFrame(data =d2)
df2

/home/beckmann/virtualenvironment/vpy3/local/lib/python3.6/site-packages/dimarray/io/nc.py:624: RuntimeWarning: 'nv4' dimension not found, define integer range
  warnings.warn(msg, RuntimeWarning)


,year,resolution,-Mx -My,-Mz,skip,time command
0,-125000,18000,-Mx 88 -My 152,-Mz 251 -Mbz 21,100,-ys -125000 -ye -25000
1,-25000,9000,-Mx 176 -My 304,-Mz 251 -Mbz 21,50,-ys -25000 -ye -5000
2,-5000,4500,-Mx 352 -My 608,-Mz 251 -Mbz 21,50,-ys -5000 -ye -1000
3,-1000,4500,-Mx 352 -My 608,-Mz 251 -Mbz 21,100,-ys -1000 -ye 0


In [5]:
ftt_file = '/home/beckmann/exp_pism/my_first_runs/input_data/ftt_18000m.nc'

In [6]:
dirs=[
    'paleo_125ka_25ka_sia_e_'+str(sia_e),
    'paleo_25ka_5ka_sia_e_'+str(sia_e),
    'paleo_5ka_1ka_sia_e_'+str(sia_e),
    'paleo_1ka_0a_sia_e_'+str(sia_e),
]

pthout = '/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/'
beds =[]
ts = []
out_state = []
out_dirs = []
for i,r in enumerate(res):
    name = 'pism_Greenland_'+str(int(r))+'m_mcb_jpl_v_no_bath.nc'
    beds.append(name)
    oname= 'gris_'+str(int(r))+'m.nc'
    name = 'ts_gris_'+str(int(r))+'m.nc'
    ts.append(pthout+dirs[i] +'/'+ name)
    name = 'gris_'+str(int(r))+'m.nc'
    out_state.append(pthout+dirs[i] +'/'+ name)
    out_dirs.append(pthout +dirs[i] +'/')
pism = '/home/beckmann/gitrepos/pism_v1.2.1/pism/bin/pismr'
input_dir = '/home/beckmann/icedata/pism_greenland_input/BedMachine/'
input_dir_pdd ='/home/beckmann/exp_pism/my_first_runs/spin_up_aschwanden/input_data/'  
CLUSTER = True

In [7]:
out_state

['/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_125ka_25ka_sia_e_3.0/gris_18000m.nc',
 '/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_25ka_5ka_sia_e_3.0/gris_9000m.nc',
 '/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_5ka_1ka_sia_e_3.0/gris_4500m.nc',
 '/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_1ka_0a_sia_e_3.0/gris_4500m.nc']

In [8]:

ptt =pthout.split('exp_pism')[0]+'mnt/cluhome/exp_pism'+pthout.split('exp_pism')[-1]
for dirname in dirs:
    if not os.path.exists(ptt + dirname ):
        os.makedirs(ptt + dirname)

In [9]:
ptt

'/home/beckmann/mnt/cluhome/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/'

In [10]:
def write_run_script(scriptname, nodes, ntask,command,Test =False):
    CLUSTER = 'True'
    if Test:
        priority = '#SBATCH --qos=priority'
        time=''
    if Test is not True:
        priority = '#SBATCH --qos=short'
        time='#SBATCH --time=23:49:00'
    bash_lines=[
    '#!/bin/bash'
    '',
    time,
    priority,
    '#SBATCH --account=anthroia',
    '#SBATCH --nodes='+str(int(nodes)),
    '#SBATCH --ntasks-per-node='+str(int(ntask)),
    '#SBATCH --job-name='+scriptname.split('.sh')[0],
    '#SBATCH --output='+scriptname.split('.sh')[0]+'.out',
    '#SBATCH --constraint=haswell',
    ]

    with open ('./scripts/'+scriptname, 'w') as rsh:
        for line in bash_lines:
            rsh.write(line + '\n')
        rsh.write('\n')
        rsh.write('\n')
        rsh.write(command)
    if CLUSTER:
        shutil.copyfile('./scripts/'+scriptname, \
                        ptt+'scripts/'+scriptname, follow_symlinks=True) 

In [11]:
i =0
def write_command1_ftt_margins(nodes, ntask,sia_e,i=0,Test=True):
    #efile = out_dirs[i]+'extra_smb_'+str(res[i])+'.nc'
    efile = out_dirs[i]+ 'check.nc'
    
    if Test:
        command_t = time_command_test[i]
        t_end = str(int(test[i]))
        #sv =''
        extra = ' -extra_times '+ str(int(times[i]))+':monthly:'+str(int(times[i])+1) +\
        ' -extra_file '+ efile +' -extra_vars climatic_mass_balance,ice_surface_temp'
        save = ' '
        
    else:
        command_t = time_command[i]
        t_end = str(int(y_end[i]))
        extra = ' -extra_times '+ str(int(y_end[i]-1))+':monthly:'+t_end+\
        ' -extra_file '+efile+' -extra_vars climatic_mass_balance,ice_surface_temp'
        
        delta_snap = int((times[i+1] -times[i])/4)
        s_times = str(int(times[i]))+':'+str(delta_snap)+':'+t_end
        save = ' -save_file '+out_dirs[i]+'snapshots.nc -save_times ' +s_times
        
    ftt_file = input_dir+'ftt_'+str(res[i])+'m.nc'
    
    #s_times = str(int(times[i]))+':500:'+str(int(times[i+1]))
    
        #sv = ' -save_file '+ts[0].split('ts_')[0]+'snapshots.nc -save_times -25000,-5000,-1000'
    #todo!
#     MAR_clim18000 = '/home/beckmann/icedata/MAR_SMB/MARv3.9/ISMIP6/proj_monthly/results/'+'clim_mean_pism_SMB_'+str(res[i])+'m.nc'
    MAR_clim18000 = '/home/beckmann/icedata/MAR_SMB/Alison/clim_mean/'+'MAR-yr-mean-1961-1990-18km_time_pism.nc'
    #MAR_clim18000 = '/home/beckmann/icedata/MAR_SMB/MARv3.9/ISMIP6/proj_monthly/results/'+'clim_mean_pism_ST_only_18000m.nc'
   
    smbflx_paleo= '-surface given,forcing,delta_T'+\
    ' -surface_delta_T_file '+ input_dir_pdd +'pism_dT.nc -sea_level constant,delta_sl -ocean_delta_sl_file '+ \
    input_dir_pdd+'pism_dSL.nc'+\
    ' -surface_given_file '+MAR_clim18000 +' -surface_given_period 1'+\
    ' -force_to_thickness_file '+ftt_file +' -surface.force_to_thickness.ice_free_alpha_factor 1000'
    
    make_faster = '-stress_balance.sia.limit_diffusivity yes -stress_balance.sia.max_diffusivity 10.0 '+\
    '-stress_balance.ssa.fd.max_speed 30.0e3'
    
   
    
    command = 'srun -n '+str(int(nodes*ntask))+' '+pism+' -i '+ input_dir+beds[i]+' -bootstrap '+ M_horizontal[i] + \
    ' ' + Mz[i] + ' -z_spacing equal -Lz 4000 -Lbz 2000 skip -skip_max ' +str(skip[0]) + ' -grid.registration center '+ command_t +\
    ' ' + smbflx_paleo + ' -options_left -stress_balance.sia.max_diffusivity 500.0'+\
     ' -calving eigen_calving,vonmises_calving,thickness_calving -eigen_calving_K 1e+16 -thickness_calving_threshold 50'+\
    ' -front_retreat_file '+input_dir + 'pism_Greenland_18000m_mcb_jpl_v_no_bath_front_retreat.nc' +\
    ' -sia_e '+str(sia_e) +\
    ' -ts_file '+ \
    ts[i]+ ' -ts_times '+ str(int(times[i]))+':yearly:'+str(int(t_end))+ \
    extra + \
    save +\
    ' -o ' + out_state[i] + ' -o_format netcdf4_parallel -o_size medium'
    return(command)

In [39]:
ts

['/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_125ka_25ka_sia_e_1.25/ts_gris_18000m.nc',
 '/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_25ka_5ka_sia_e_1.25/ts_gris_9000m.nc',
 '/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_5ka_1ka_sia_e_1.25/ts_gris_4500m.nc',
 '/home/beckmann/exp_pism/alison/spin_up/PISM_1.2.1/MARv3.11.2_CESM2/paleo_1ka_0a_sia_e_1.25/ts_gris_4500m.nc']

In [15]:
Test = False

nodes = 8
ntask = 16
command = write_command1_ftt_margins(nodes,ntask,sia_e,0,Test)
write_run_script('run1_paleo125k_sia_e'+str(sia_e)+'.sh', nodes, ntask,command, Test)

## run 2

In [12]:
regrid_command =' -regrid_vars litho_temp,enthalpy,age,tillwat,bmelt,Href,thk'
def write_command2_ftt_margins(nodes, ntask,sia_e,i=1,Test=True):
    #efile = out_dirs[i]+'extra_smb_'+str(res[i])+'.nc'
    efile = out_dirs[i]+ 'check.nc'
    
    if Test:
        command_t = time_command_test[i]
        t_end = str(int(test[i]))
        #sv =''
        extra = ' -extra_times '+ str(int(times[i]))+':monthly:'+str(int(times[i])+1) +\
        ' -extra_file '+ efile +' -extra_vars climatic_mass_balance,ice_surface_temp'
        save = ' '
        
    else:
        command_t = time_command[i]
        t_end = str(int(y_end[i]))
        extra = ' -extra_times '+ str(int(y_end[i]-1))+':monthly:'+t_end+\
        ' -extra_file '+efile+' -extra_vars climatic_mass_balance,ice_surface_temp'
        
        delta_snap = int((times[i+1] -times[i])/4)
        s_times = str(int(times[i]))+':'+str(delta_snap)+':'+t_end
        save = ' -save_file '+out_dirs[i]+'snapshots.nc -save_times ' +s_times
        
    ftt_file = input_dir+'ftt_'+str(res[i])+'m.nc'
    
    #s_times = str(int(times[i]))+':500:'+str(int(times[i+1]))
    
        #sv = ' -save_file '+ts[0].split('ts_')[0]+'snapshots.nc -save_times -25000,-5000,-1000'
    #todo!
    MAR_clim18000 = '/home/beckmann/icedata/MAR_SMB/Alison/clim_mean/'+'MAR-yr-mean-1961-1990-9km_time_pism.nc'
    
   
    smbflx_paleo= '-surface given,forcing,delta_T'+\
    ' -surface_delta_T_file '+ input_dir_pdd +'pism_dT.nc -sea_level constant,delta_sl -ocean_delta_sl_file '+ \
    input_dir_pdd+'pism_dSL.nc'+\
    ' -surface_given_file '+MAR_clim18000 +'  -surface_given_period 1'+\
    ' -force_to_thickness_file '+ftt_file +' -surface.force_to_thickness.ice_free_alpha_factor 1000'
    
    make_faster = '-stress_balance.sia.limit_diffusivity yes -stress_balance.sia.max_diffusivity 10.0 '+\
    '-stress_balance.ssa.fd.max_speed 30.0e3'
    
   
    
    command = 'srun -n '+str(int(nodes*ntask))+' '+pism+' -bootstrap  -i '+ input_dir+beds[i]+' -regrid_file '+ out_state[i-1] \
    +' '+ regrid_command+' ' + M_horizontal[i] + \
    ' ' + Mz[i] + ' -z_spacing equal -Lz 4000 -Lbz 2000 skip -skip_max ' +str(skip[0]) + ' -grid.registration center '+ command_t +\
    ' ' + smbflx_paleo + ' -options_left -stress_balance.sia.max_diffusivity 500.0'+\
     ' -calving eigen_calving,vonmises_calving,thickness_calving -eigen_calving_K 1e+16 -thickness_calving_threshold 50'+\
    ' -front_retreat_file '+input_dir + 'pism_Greenland_18000m_mcb_jpl_v_no_bath_front_retreat.nc' +\
    ' -sia_e '+str(sia_e) +\
    ' -ts_file '+ \
    ts[i]+ ' -ts_times '+ str(int(times[i]))+':yearly:'+str(int(t_end))+ \
    extra + \
    save +\
    ' -o ' + out_state[i] + ' -o_format netcdf4_parallel -o_size medium'
    return(command)

In [15]:
Test = False
i=1

nodes = 16
ntask = 16
command = write_command2_ftt_margins(nodes,ntask,sia_e,i,Test)
write_run_script('run2_paleo_25ka_5ka_sia_e'+str(sia_e)+'.sh', nodes, ntask,command, Test)